In [7]:
import tensorflow as tf
print(tf.__version__)
import os
import zipfile
import random
from shutil import copyfile, move

2.16.1


In [8]:
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip -O rockpaperscissors.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
with zipfile.ZipFile('rockpaperscissors.zip', 'r') as zip_ref:
    zip_ref.extractall('rockpaperscissors')

In [14]:
base_dir = 'rockpaperscissors/rockpaperscissors'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

categories = ['rock', 'paper', 'scissors']

for category in categories:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)


In [15]:
for category in categories:
    category_path = os.path.join(base_dir, category)
    files = os.listdir(category_path)
    random.shuffle(files)

    train_size = int(0.6 * len(files))
    train_files = files[:train_size]
    validation_files = files[train_size:]

    for file in train_files:
        move(os.path.join(category_path, file), os.path.join(train_dir, category, file))

    for file in validation_files:
        move(os.path.join(category_path, file), os.path.join(validation_dir, category, file))


In [16]:
for category in categories:
    train_files = os.listdir(os.path.join(train_dir, category))
    validation_files = os.listdir(os.path.join(validation_dir, category))
    print(f"Category: {category}")
    print(f"Training files: {len(train_files)}")
    print(f"Validation files: {len(validation_files)}\n")


Category: rock
Training files: 435
Validation files: 291

Category: paper
Training files: 427
Validation files: 285

Category: scissors
Training files: 450
Validation files: 300



In [17]:
train_dir = 'rockpaperscissors/rockpaperscissors/train'
validation_dir = 'rockpaperscissors/rockpaperscissors/validation'

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
test_datagen = ImageDataGenerator(
                    rescale=1./255)

In [20]:
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=4,
        class_mode='categorical')
 
validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='categorical')

Found 1312 images belonging to 3 classes.
Found 876 images belonging to 3 classes.


In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(len(categories), activation='softmax')
])


c:\Users\Mull\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 148, 148, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 74, 74, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 72, 72, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 15, 15, 512)    │       590,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,530,691 (51.62 MB)

 Trainable params: 13,530,691 (51.62 MB)

 Non-trainable params: 0 (0.00 B)

In [29]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [30]:
model.fit(
      train_generator,
      steps_per_epoch=25,  # berapa batch yang akan dieksekusi pada setiap epoch
      epochs=20, # tambahkan epochs jika akurasi model belum optimal
      validation_data=validation_generator, # menampilkan akurasi pengujian data validasi
      validation_steps=5,  # berapa batch yang akan dieksekusi pada setiap epoch
      verbose=2)

Epoch 1/20


c:\Users\Mull\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 - 6s - 235ms/step - accuracy: 0.3500 - loss: 0.6787 - val_accuracy: 0.3000 - val_loss: 0.6440
Epoch 2/20
25/25 - 4s - 169ms/step - accuracy: 0.4300 - loss: 0.6231 - val_accuracy: 0.6000 - val_loss: 0.5884
Epoch 3/20
25/25 - 4s - 167ms/step - accuracy: 0.4200 - loss: 0.6233 - val_accuracy: 0.2500 - val_loss: 0.5950
Epoch 4/20
25/25 - 4s - 180ms/step - accuracy: 0.4600 - loss: 0.6262 - val_accuracy: 0.3500 - val_loss: 0.6423
Epoch 5/20
25/25 - 5s - 183ms/step - accuracy: 0.5500 - loss: 0.5636 - val_accuracy: 0.6000 - val_loss: 0.5240
Epoch 6/20
25/25 - 4s - 173ms/step - accuracy: 0.5500 - loss: 0.5617 - val_accuracy: 0.6500 - val_loss: 0.4277
Epoch 7/20
25/25 - 4s - 168ms/step - accuracy: 0.6900 - loss: 0.4311 - val_accuracy: 0.8500 - val_loss: 0.3186
Epoch 8/20
25/25 - 4s - 174ms/step - accuracy: 0.8600 - loss: 0.2945 - val_accuracy: 0.8500 - val_loss: 0.3327
Epoch 9/20
25/25 - 4s - 167ms/step - accuracy: 0.8600 - loss: 0.2452 - val_accuracy: 0.9500 - val_loss: 0.1468
Epoch 10/20


c:\Users\Mull\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


25/25 - 4s - 171ms/step - accuracy: 0.9000 - loss: 0.1438 - val_accuracy: 0.8500 - val_loss: 0.2114
Epoch 16/20
25/25 - 4s - 164ms/step - accuracy: 0.9400 - loss: 0.0922 - val_accuracy: 0.9000 - val_loss: 0.0858
Epoch 17/20
25/25 - 4s - 166ms/step - accuracy: 0.8800 - loss: 0.2548 - val_accuracy: 0.9500 - val_loss: 0.0973
Epoch 18/20
25/25 - 4s - 164ms/step - accuracy: 0.9100 - loss: 0.2011 - val_accuracy: 0.9500 - val_loss: 0.1238
Epoch 19/20
25/25 - 4s - 166ms/step - accuracy: 0.8800 - loss: 0.1918 - val_accuracy: 1.0000 - val_loss: 0.0367
Epoch 20/20
25/25 - 4s - 161ms/step - accuracy: 0.8900 - loss: 0.1759 - val_accuracy: 0.9000 - val_loss: 0.1715


In [31]:
evaluation = model.evaluate(validation_generator)
print("Validation Accuracy: {:.2f}%".format(evaluation[1] * 100))

219/219 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9172 - loss: 0.1730
Validation Accuracy: 91.10%


In [ ]:
from google.colab import files
from keras.preprocessing import image
import numpy as np

uploaded = files.upload()

for filename in uploaded.keys():
    path = '/content/' + filename
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes)
    print(classes[0])
    print(categories[np.argmax(classes[0])])
